In [1]:
# !pip install sympy

In [58]:
from sympy import symbols, cos, sin, Matrix, diag, zeros, eye, pi
from sympy.physics.mechanics import dynamicsymbols, mechanics_printing
mechanics_printing()

# Symbolic functions

In [59]:
def skew(v):
    return Matrix([[0, -v[2], v[1]],
                   [v[2], 0, -v[0]],
                   [-v[1], v[0], 0]])

In [60]:
def exp3(omega, theta):
    omega = skew(omega)
    R = eye(3) + sin(theta) * omega + (1 - cos(theta)) * omega * omega
    return R

In [61]:
def exp6(twist, theta):
    omega = skew(twist[:3])
    v = Matrix(twist[3:])
    T = eye(4)
    T[:3, :3] = exp3(twist[:3], theta)
    T[:3, 3] = (eye(3) * theta + (1 - cos(theta)) * omega +
                (theta - sin(theta)) * omega * omega) * v
    return T

In [62]:
def Ad(T):
    AdT = zeros(6)
    R = Matrix(T[:3, :3])
    AdT[:3, :3] = R
    AdT[3:, 3:] = R
    AdT[3:, :3] = skew(T[:3, 3]) * R
    return AdT

# 5.1.1 Space Jacobian
## Example: Scara Robot (RRRP)


### Define symbols

In [63]:
th1, th2, th3, th4 = dynamicsymbols("theta_1, theta_2, theta_3, theta_4")
L1, L2, L3, L4 = symbols("L_1, L_2, L_3, L_4")



### Define screw axes

In [70]:
import numpy as np

S1 = Matrix([[0],[0],[1],[0],[0],[0]])
S2 = Matrix([[0],[0],[1],[0],[-L1],[0]])
S3 = Matrix([[0],[0],[1],[0],[-L1-L2],[0]])
S4 = Matrix([[0],[0],[1],[0],[-L1-L2-L3],[0]])
th1=0
th2=0
th3=pi/2
th4=pi/2
f = Matrix([10, 10, 0])
m = Matrix([0, 0, 10])

### Define columns of Jacobian matrix

In [71]:
Js1 = S1

In [72]:
Js2 = Ad(exp6(S1, th1)) * S2
Js2.simplify()
# in sympy unlike numpy, * is the matrix multiplication and not @

In [73]:
Js3 = Ad(exp6(S1, th1) * exp6(S2, th2)) * S3
Js3.simplify() #questa funzione cerca di semplificare al massimo la formula, facendo ciò otteniamo la stessa fomula che è riportata sul libro MR
Js3

⎡   0    ⎤
⎢        ⎥
⎢   0    ⎥
⎢        ⎥
⎢   1    ⎥
⎢        ⎥
⎢   0    ⎥
⎢        ⎥
⎢-L₁ - L₂⎥
⎢        ⎥
⎣   0    ⎦

In [74]:
Js4 = Ad(exp6(S1, th1) * exp6(S2, th2) * exp6(S3, th3)) * S4
Js4.simplify()
Js4

⎡   0    ⎤
⎢        ⎥
⎢   0    ⎥
⎢        ⎥
⎢   1    ⎥
⎢        ⎥
⎢   L₃   ⎥
⎢        ⎥
⎢-L₁ - L₂⎥
⎢        ⎥
⎣   0    ⎦

In [69]:
L1, L2, L3, L4 = symbols("L_1, L_2, L_3, L_4")

B1 = Matrix([[0],[0],[1],[0],[L1+L2+L3+L4],[0]])
B2 = Matrix([[0],[0],[1],[0],[L2+L3+L4],[0]])
B3 = Matrix([[0],[0],[1],[0],[L3+L4],[0]])
B4 = Matrix([[0],[0],[1],[0],[L4],[0]])

In [49]:

Jb4 = B4
Jb3 = Ad(exp6(B4, th4)) * B3
Jb2 = Ad(exp6(B4, th4) * exp6(B3, th3)) * B2
Jb1 = Ad(exp6(B4, th4) * exp6(B3, th3) * exp6(B2, th2)) * B1
Jb4.simplify()
Jb3.simplify()
Jb2.simplify()
Jb1.simplify()


In [28]:
Jb = zeros(6, 4)
Jb[:, 0] = Jb1
Jb[:, 1] = Jb2
Jb[:, 2] = Jb3
Jb[:, 3] = Jb4
Jb

⎡                           0                                              0  
⎢                                                                             
⎢                           0                                              0  
⎢                                                                             
⎢                           1                                              1  
⎢                                                                             
⎢  -L₁⋅sin(θ₂ + θ₃ + θ₄) - L₂⋅sin(θ₃ + θ₄) - L₃⋅sin(θ₄)      -L₂⋅sin(θ₃ + θ₄) 
⎢                                                                             
⎢L₁⋅cos(θ₂ + θ₃ + θ₄) + L₂⋅cos(θ₃ + θ₄) + L₃⋅cos(θ₄) + L₄  L₂⋅cos(θ₃ + θ₄) + L
⎢                                                                             
⎣                           0                                              0  

                       0         0 ⎤
                                   ⎥
                       0         0 ⎥
                   

### Fill in columns of Jacobian

In [75]:
Js = zeros(6, 4)
Js

⎡0  0  0  0⎤
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎣0  0  0  0⎦

In [76]:
Js[:, 0] = Js1
Js[:, 1] = Js2
Js[:, 2] = Js3
Js[:, 3] = Js4
Js

⎡0   0      0         0    ⎤
⎢                          ⎥
⎢0   0      0         0    ⎥
⎢                          ⎥
⎢1   1      1         1    ⎥
⎢                          ⎥
⎢0   0      0         L₃   ⎥
⎢                          ⎥
⎢0  -L₁  -L₁ - L₂  -L₁ - L₂⎥
⎢                          ⎥
⎣0   0      0         0    ⎦